In [7]:
!pip install hyperopt

In [18]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
cd "/content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/Matrix_3_road_sign/data"

/content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/Matrix_3_road_sign/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, param_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model. fit( 
      X_train,
      y_train,
      batch_size=param_fit.get('batch_size', 128),
      epochs=param_fit.get('epochs',5),
      verbose=param_fit.get('verbose', 1),
      validation_data=param_fit.get('validation_data', (X_train, y_train)), 
      callbacks=[tensorboard_callback]
      
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  
  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [16]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential ([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
                     
                     ])

train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.8977 - accuracy: 0.2325 - val_loss: 1.4064 - val_accuracy: 0.5372
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 1.1271 - accuracy: 0.6355 - val_loss: 0.3472 - val_accuracy: 0.8826
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4974 - accuracy: 0.8415 - val_loss: 0.1317 - val_accuracy: 0.9586
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3018 - accuracy: 0.9079 - val_loss: 0.0597 - val_accuracy: 0.9823
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2230 - accuracy: 0.9309 - val_loss: 0.0418 - val_accuracy: 0.9888


0.9607709750566893

In [0]:
def get_model(params):
  return Sequential ([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),

    Dense(num_classes, activation='softmax')
                     
    ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model. fit( 
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=params.get('epochs',5),
      verbose=0      
  )
  
  accuracy = model.evaluate(X_test, y_test)[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [34]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0, 1.0),
    'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0, 1.0),
    'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0, 1.0),
    'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0, 1.0),
    'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0, 1.0),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 4.9361 - accuracy: 0.0000e+00

 16/138 [==>...........................]
 - ETA: 0s - loss: 3.7649 - accuracy: 0.0000e+00

 27/138 [====>.........................]
 - ETA: 0s - loss: 3.8875 - accuracy: 0.0000e+00

 42/138 [========>.....................]
 - ETA: 0s - loss: 3.7655 - accuracy: 0.0000e+00

 57/138 [===========>..................]
 - ETA: 0s - loss: 3.7566 - accuracy: 0.0000e+00

 71/138 [==============>...............]
 - ETA: 0s - loss: 3.5773 - accuracy: 0.0951    
